
# Prever se uma pessoa ficará inadimplente em um futuro próximo

![image-2.png](attachment:image-2.png)

![Imagem_ComparandoCassificadores.png](attachment:Imagem_ComparandoCassificadores.png)

## Segmento Cartão de Crédito
#### A área de crédito deseja avaliar a propensão ao risco de seus clientes e implementar políticas de redução da inadimplência
#### Dicionário de variáveis¶


![image.png](attachment:image.png)

![image.png](attachment:image.png)

In [ ]:
# Importar as bibliotecas
#!install seaborn
! pip install seaborn

#!install scikit-learn
! pip install scikit-learn


import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import sklearn

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier,export_graphviz
from sklearn import preprocessing
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve

# elimina os warnings das bibliotecas
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Importar os dados
Base_Inad = pd.read_csv("Inadimplencia_quantitativa_2.csv", sep=";")

print("Total number of transactions: "+ str(Base_Inad.shape[0]))
Base_Inad.head(5)


In [ ]:
Base_Inad.info()

In [ ]:
pd.DataFrame(zip(Base_Inad.isnull().sum(), Base_Inad.isnull().sum()/len(Base_Inad)), columns=['Count', 'Proportion'], index=Base_Inad.columns)


In [ ]:
Base_Inad.describe()

In [ ]:
# Tabela de frequência da variável Resposta do modelo
Resposta =pd.pivot_table(Base_Inad, index='Resposta', values='cliente', aggfunc=np.count_nonzero)
Resposta['%'] = ((pd.pivot_table(Base_Inad, index='Resposta', values = 'cliente', aggfunc=np.count_nonzero)/Base_Inad['cliente'].count())*100).round(2)

Resposta

In [ ]:
# Como estamos construindo um modelo para prever a Inadimplência de um cliente da rede:0=Adimplente 1=Inadimplente
# Para ter certeza de que é uma variável binária, vamos usar a função countplot () do Seaborn.

sb.countplot(x='Resposta',data=Base_Inad, palette='hls');

In [ ]:
# Analisando as variáveis preditoras

# Variável preditora: Atrasos

sb.histplot(data=Base_Inad, x="Atrasos");

In [ ]:
# Neste estudo o foco é a Inadimplência, assim....

sb.boxplot(x='Resposta', y='Atrasos', data=Base_Inad, palette='hls')

In [ ]:
sb.boxplot(x='Resposta', y='TempoRel', data=Base_Inad, palette='hls');

In [ ]:
# transformando a varíavel tempo de relacionamneto em anos
Base_Inad['TempoRel'] = Base_Inad['TempoRel']/365
Base_Inad.describe()

In [ ]:
sb.boxplot(x='Resposta', y='valorFatura', data=Base_Inad, palette='hls')

In [ ]:
# Uma opção sobre como eliminar outliers é achatamento de algum valor descrepante

Base_Inad.loc[Base_Inad['valorFatura']>=50000, 'valorFatura'] = 10000

In [ ]:
sb.boxplot(x='Resposta', y='valorFatura', data=Base_Inad, palette='hls')

In [ ]:
sb.boxplot(x='Resposta', y='GastosAlim', data=Base_Inad, palette='hls');

In [ ]:
sb.boxplot(x='Resposta', y='RendaMensal', data=Base_Inad, palette='hls');

In [ ]:
# comparison between Regrisk and target

RegRisc_cat = pd.crosstab(Base_Inad['Resposta'],Base_Inad['RegRisc'])
RegRisc_cat.div(RegRisc_cat.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, figsize=(6, 6));

In [ ]:
# Converting categorical variables to a dummy indicators
RgRisk = pd.get_dummies(Base_Inad['RegRisc'],drop_first=True)
RgRisk.head()

In [ ]:
# Alterando os dados
Base_Inad.drop(['cliente','RegRisc'],axis=1,inplace=True)
Base_Inad.head()

In [ ]:
Base_Inad_dmy = pd.concat([Base_Inad,RgRisk],axis=1)
Base_Inad_dmy.head()

In [ ]:
Base_Inad_dmy.info()

In [ ]:
# 1º: Separar o conjunto em variavel resposta e variaveis de treinamento

X = Base_Inad_dmy.iloc[:,[1,2,3,4,5,6,7,8]].values
y = Base_Inad_dmy.iloc[:,0].values
X.shape, y.shape


In [ ]:
# Agora dividir em treino e teste (teste com 30%)


# Dividindo os dados em treino e teste:
X_train, X_test, y_train, y_test = train_test_split(Base_Inad_dmy.drop('Resposta',axis=1),Base_Inad_dmy['Resposta'],test_size=0.3, random_state=17)


#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .3, random_state=25)

X_train.shape, X_test.shape

In [ ]:
 y_train.shape, y_test.shape

In [ ]:
 y_test.head()

In [ ]:
X_train.describe()

# Magnitude das variáveis preditoras:
    
    Em função das características de alguns algoritmos há necessidade de realizar transfomação nas variáveis preditoras.

![image.png](attachment:image.png)

In [ ]:
# Feature Scaling
# Only on Independent Variable

from sklearn.preprocessing import MinMaxScaler


MinMax = MinMaxScaler()
X_train = MinMax.fit_transform(X_train)

X_test = MinMax.fit_transform(X_test)
X_train = pd.DataFrame(X_train, columns=['Atrasos','TempoRel','valorFatura','GastosAlim','RendaMensal','II','III','IV'])
X_train.head()



# 1.o MODELO: ÁRVORE DE DECISÃO

![image.png](attachment:image.png)

    Esses modelos de aprendizado supervisionado são empregados em tarefas tanto de regressão quanto classificação.
    O objetivo do método é criar uma árvore que aprende com os dados por meio de regras básicas, chamadas de if-then rules.
    O modelo identifica qual é o melhor atributo para separar os dados utilizando o critério escolhido, que, neste caso, pode ser entropy ou gini.

https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html


![image.png](attachment:image.png)

In [ ]:
# Criando modelo

Classif_tree = DecisionTreeClassifier()


#### Treinando o modelo de arvore de decisão:

In [ ]:
classif = Classif_tree.fit(X_train, y_train)

# Como treinamos a árvore sem especificar nenhum parâmetro,
# ela cresceu de forma indefinida e ficou grande demais impossibilitando a visualização da mesma.


In [ ]:
# Verificando as features mais importantes para o modelo treinado:
classif.feature_importances_

# O código acima nos retorna um array com o valor de cada variável:

In [ ]:
for feature,importancia in zip(X_train.columns,classif.feature_importances_):
    print("{}:{}".format(feature, importancia))

In [ ]:
# Testando o modelo
y_pred_all = classif.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
# Matriz de Confusão

confusion_matrix = confusion_matrix(y_test, y_pred_all)
confusion_matrix


In [ ]:
print(classification_report(y_test, y_pred_all))

# Resultado do classification_report:

#### Parâmetros do classificador DecisionTreeClassifier

In [ ]:
# criterion: medida de qualidade da divisão, aqui que definimos qual dessas métricas (Indice Gini, Chi-Square, Information Gain ou a redução da variância) iremos usar.
# splitter: estratégia utilizada para dividir o nó de decisão.
# max_depth: profundidade máxima da árvore.
# min_samples_split: número de amostras mínimas para considerar um nó para divisão.
# min_samples_leaf: número de amostras mínimas no nível folha.
# clf = DecisionTreeClassifier(max_leaf_nodes=10, random_state=0)

# clf = DecisionTreeClassifier(max_depth=5, criterion='gini', random_state=0)

In [ ]:
clf = DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=5,
            max_leaf_nodes=None,
            min_impurity_decrease=0.0,
            min_samples_leaf=10, min_samples_split=10,
            min_weight_fraction_leaf=0.0,
            random_state=17, splitter='best')

clf.fit(X_train, y_train)

In [ ]:
# Verificando as features mais importantes para o modelo treinado:
clf.feature_importances_

# O código acima nos retorna um array com o valor de cada variável:

In [ ]:
for feature,importancia in zip(X_train.columns,clf.feature_importances_):
    print("{}:{}".format(feature, importancia))

In [ ]:
# Testando o modelo
y_pred = clf.predict(X_test)

In [ ]:
y_pred

# O código acima nos da a seguinte saída:

### Predição da Árvore de Decisão

![image.png](attachment:image.png)

### Precision score = VP/(FP+VP))
### Recall score = VP/(FN+VP)
### F1 Score = 2* Precision Score * Recall Score/ (Precision Score + Recall Score)

link: https://vitalflux.com/accuracy-precision-recall-f1-score-python-example/

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
confusion_matrix


In [ ]:
print(classification_report(y_test, y_pred))

# Resultado do classification_report:

In [ ]:
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
fig = plt.figure(figsize=(16,12))
a = plot_tree(clf, feature_names=['Atrasos', 'TempoRel', 'valorFatura', 'GastosAlim', 'RendaMensal', 'II',
       'III', 'IV'], fontsize=12,
filled=True,
class_names=['Inad(não)', 'Inad(sim)'])

# 2.o MODELO: REGRESSÃO LOGÍSTICA

![image.png](attachment:image.png)

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [ ]:
# Agora vamos refazer o modelo utilizando scikit-learn para maior facilidade de obter as métricas de ajuste.
# Agora vamos fazer com sklearn para aproveitar as métricas
#Predicting using Logistic Regression for Binary classification


logreg = LogisticRegression(penalty='none', solver='newton-cg',random_state=17)


In [ ]:
#Em seguida, você precisa treinar o seu modelo. Precisamos rodar a regressão para X_train e y_train:

# treina o modelo
logistica = logreg.fit(X_train, y_train)  #fitting the model

In [ ]:
print(logistica.coef_)

In [ ]:
cols = X_train.columns


In [ ]:
# Os coeficientes do modelo (b1,b2,b3....)

coefficients = pd.concat([pd.DataFrame(cols),pd.DataFrame(np.transpose(logistica.coef_))], axis = 1)
coefficients

In [ ]:
X_test

In [ ]:
# faz predicao e salva em y_pred
# y_pred_log = logreg.predict(X_test)
y_pred_log = logistica.predict(X_test)
y_pred_log

In [ ]:

# acuracia
logistica.score(X_test, y_test)


In [ ]:
# matriz de confusao
from sklearn.metrics import confusion_matrix

confusion_matrix_log = confusion_matrix(y_test, y_pred_log)
confusion_matrix_log

In [ ]:
 print(classification_report(y_test, logistica.predict(X_test)))

In [ ]:
# Predizendo as probabilidades
yhat = logistica.predict_proba(X_test) #prediction
yhat

In [ ]:
y_pred_prob = yhat[:, 1] # manter somente para a classe positiva
y_pred_prob

# Modelos Ensemble


![image.png](attachment:image.png)

    Tipos de Modelos Ensemble
    Características Principais:
    Bagging geralmente usa mesmo tipo de modelos individuais, cada um de forma independente em relação ao outro,
    de forma paralela.
    O algoritmo final é então feito a partir de algum tipo de resultado médio do que foi obtido a partir dos modelos bases.

    Boosting geralmente usa mesmo tipo de modelos individuais, que são aplicados de forma sequencial (o posterior depende do antecessor) e depois combinados no modelo final.

    Stacking geralmente usa tipos diferentes de modelos individuais, treinando-os em paralelo.
    É então aplicado um modelo no output dos weak learners (podendo incluir ou não as features utilizadas para treiná-los).

# 3.o MODELO: Random Forest

![image.png](attachment:image.png)

In [ ]:
# Documentação https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
from sklearn.ensemble import RandomForestClassifier

random_forest_model = RandomForestClassifier(n_estimators=5, max_depth=3, n_jobs=-1,random_state=17)
random_forest_model.fit(X_train, y_train)



In [ ]:
y_predito = random_forest_model.predict(X_test)


In [ ]:
# matriz de confusao
from sklearn.metrics import confusion_matrix

confusion_matrix_RF = confusion_matrix(y_test, random_forest_model.predict(X_test))
confusion_matrix_RF

In [ ]:
for caracteristica, score in zip(X_train.columns, random_forest_model.feature_importances_):
  print('caracteristica {} e seu score {}'.format(caracteristica, score))

In [ ]:
print(classification_report(y_test, random_forest_model.predict(X_test)))

# 4.o MODELO: Gradient Boosting¶

O Gradient Boosting é um outro tipo de algoritmo de Boosting.

Técnica de aprendizado de máquina para problemas de regressão e classificação, que produz um modelo de previsão na forma de um conjunto de modelos simples (baixa previsão), geralmente árvores de decisão.

Ele constrói o modelo em etapas. O objetivo do algoritmo é criar uma corrente de modelos fracos, onde cada um tem como objetivo minimizar o erro do modelo anterior, por meio de uma função de perda.

os erros são minimizado pelo algoritmo de gradiente descendente(gradient descending).

Aos ajustes de cada modelo fraco é multiplicado um valor chamado de taxa de aprendizagem. Esse valor, tem como objetivo determinar o impacto de cada árvore no modelo final. Quanto menor o valor, menor a contribuição de cada árvore.

In [ ]:
# Documentação https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
from sklearn.ensemble import GradientBoostingClassifier


In [ ]:
gradient_boost = GradientBoostingClassifier(random_state=17)

gradient_boost.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix


# Matriz de Confusão

confusion_matrix = confusion_matrix(y_test, gradient_boost.predict(X_test))
confusion_matrix

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, gradient_boost.predict(X_test)))

# Resultado do classification_report:

# 5.o MODELO:  KNN K-Nearest Neighbors

![image.png](attachment:image.png)

In [ ]:
#https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html

from sklearn.neighbors import KNeighborsClassifier


KNN_classificador = KNeighborsClassifier(n_neighbors = 101)

KNN_classificador.fit(X_train, y_train)




In [ ]:
# matriz de confusao
from sklearn.metrics import confusion_matrix

confusion_matrix_KNN = confusion_matrix(y_test, KNN_classificador.predict(X_test))
confusion_matrix_KNN

In [ ]:
print(classification_report(y_test, KNN_classificador.predict(X_test)))

# 6.o MODELO: SVM Support Vector Machine


![image.png](attachment:image.png)

In [ ]:
# Documentação de referência: https://scikit-learn.org/stable/modules/classes.html#module-sklearn.svm
from sklearn.svm import LinearSVC

modelo_SVM = LinearSVC(random_state=1234)
modelo_SVM.fit(X_train, y_train)

In [ ]:
y_predito = modelo_SVM.predict(X_test)

In [ ]:
# matriz de confusao

confusion_matrix_SVM = confusion_matrix(y_test, y_predito)
confusion_matrix_SVM

In [ ]:
from sklearn import metrics

print("Acurácia: {}".format(metrics.accuracy_score(y_test, y_predito)))

In [ ]:
 print(classification_report(y_test, modelo_SVM.predict(X_test)))